In [22]:
import tensorflow as tf 
from keras.models import Model
import os
import os
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.decomposition import PCA
import random
from scipy.spatial import distance
from pprint import pprint

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/challenge_test_data.zip

In [ ]:
model = tf.keras.applications.efficientnet.EfficientNetB7(weights='imagenet', include_top=True)

In [ ]:
model.summary()

In [ ]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("avg_pool").output)


GALLERY_PATH = '/content/gallery/'
QUERY_PATH = '/content/query/'

In [ ]:
import utils
target_shape = (256, 256)

gallery = utils.Dataset(data_path=GALLERY_PATH).get_dataset()
query = utils.Dataset(data_path=QUERY_PATH).get_dataset()

In [ ]:
import utils
gallery_features, gallery_urls, gallery_labels = utils.compute_features(gallery, feat_extractor)
query_features, query_urls, query_labels = utils.compute_features(query, feat_extractor)

results_eu = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls)
results_cos = utils.compute_results(query_features, gallery_features, query_urls, gallery_urls, dist= 'cosine')

In [ ]:
import requests
import json


def submit(results, url="http://tinyurl.com/IML2022"):
    res = json.dumps(results)
    response = requests.post(url, res)
    try:
        result = json.loads(response.text)
        print(f"accuracy is {result['results']}")
    except json.JSONDecodeError:
        print(f"ERROR: {response.text}")



mydata = dict()
mydata['groupname'] = "The Ostriches"

mydata["images"] = results_eu
submit(mydata)

mydata["images"] = results_cos
submit(mydata)

accuracy is [0.5846153846153846, 0.676923076923077, 0.676923076923077